# Lesson 7.1: Building a Multi-functional Chatbot

---

In previous Modules, we learned about the core components of LangChain such as LLMs, Prompts, Chains, Agents, Tools, and Memory. Now it's time to combine all that knowledge to build a practical and useful application: a **multi-functional chatbot**. This chatbot will not only be able to chat but also perform tasks like searching for information, performing calculations, and interacting with external APIs.

This lesson will focus on designing the architecture for such a chatbot, how to use an Agent to orchestrate functions, and how to integrate a simple user interface.

## 1. Designing the Architecture for a Multi-functional Chatbot

To build a chatbot capable of performing various types of tasks, we need a flexible architecture where the Large Language Model (LLM) acts as the "brain" and other components extend its capabilities.

* **LLM (Large Language Model):** The core of the chatbot, responsible for understanding natural language, reasoning, and generating responses.
* **Agent:** Acts as the intelligent "orchestrator." Instead of just answering, the Agent decides whether it needs to use a specific tool to fulfill the user's request.
* **Tools:** Specific functions that the Agent can call to interact with the external world or perform specialized tasks. For example:
    * **Web Search:** To look up current information or factual data.
    * **Calculation:** To solve arithmetic problems.
    * **API Interaction:** To retrieve data from external services (weather, quotes, task management, etc.).
    * **Knowledge Base Query (RAG):** To answer questions based on internal documents.
* **Memory:** Helps the chatbot maintain conversation context across multiple turns, allowing for more natural and seamless interactions.
* **User Interface (UI):** Provides a way for users to interact with the chatbot (e.g., web interface, mobile application).




---

## 2. Using an Agent with Multiple Tools

The key to creating a multi-functional chatbot is to equip the Agent with a diverse set of Tools and provide it with the ability to select the appropriate Tool based on the user's intent.

* **Clear Tool Definitions:** Each Tool needs a unique `name` and a clear `description` outlining its function and the type of input it expects. The LLM will use this `description` to decide when to call the Tool.
* **Tool Collection:** Place all Tools that the Agent can use into a list.
* **Agent Prompt:** The Agent's prompt needs to instruct the LLM on how to think (Thought), act (Action), and observe (Observation) to use the Tools effectively. It also needs a placeholder for `agent_scratchpad` to track the Agent's intermediate steps.
* **Memory Integration:** For the Agent to remember previous conversation turns, we will use a type of Memory (e.g., `ConversationBufferMemory`) and integrate it into the Agent's prompt via `MessagesPlaceholder`.


---

## 3. Integrating a Simple User Interface

For users to interact with the chatbot, we need an interface. For Python applications, **Streamlit** and **Gradio** are two excellent frameworks for building web interfaces quickly and easily.

* **Streamlit:**
    * **Pros:** Pure Python syntax, very fast for developing interactive data and ML applications. Supports various UI widgets, flexible layouts.
    * **When to use:** Building dashboards, demo applications, or internal tools.
* **Gradio:**
    * **Pros:** Extremely simple to create interfaces for ML models and Python functions. Just a few lines of code can produce an interface.
    * **When to use:** Quickly sharing models, creating simple demos.

In this practical example, we will use **Streamlit** to build our chatbot's interface, as it offers better flexibility for a full-fledged chatbot application.


---

## 4. Practical Example: Building and Running a Multi-functional Chatbot

We will build a Streamlit chatbot with the following functionalities:
* **Web Search:** Using `SerpAPIWrapper`.
* **Calculation:** Using the `Calculator` Tool.
* **Weather Lookup:** Using a simple Custom Tool (as built in previous lessons).
* **Context Preservation:** Using `ConversationBufferMemory`.

**Project Structure:**
You will create a single Python file, for example `app.py`.

**Preparation:**
* Ensure you have the necessary libraries installed: `langchain-openai`, `google-search-results`, `numexpr`, `streamlit`, `requests`.
* Set the `OPENAI_API_KEY` and `SERPAPI_API_KEY` environment variables.

In [ ]:
import os
import streamlit as st
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.utilities import SerpAPIWrapper
from langchain_community.tools import Tool
from langchain_community.tools.calculator.tool import Calculator
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain.memory import ConversationBufferMemory
import requests
import json # To handle JSON from weather API (if any)

# Thiết lập biến môi trường cho khóa API của OpenAI và SerpAPI
# Đảm bảo bạn đã đặt các biến này trong môi trường của mình hoặc trong Streamlit Secrets
# os.environ["OPENAI_API_KEY"] = st.secrets["OPENAI_API_KEY"]
# os.environ["SERPAPI_API_KEY"] = st.secrets["SERPAPI_API_KEY"]

# --- 1. Khởi tạo LLM ---
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# --- 2. Định nghĩa các Tools ---

# Công cụ tìm kiếm web
search_tool = Tool(
    name="Google Search",
    func=SerpAPIWrapper().run,
    description="Hữu ích khi bạn cần tìm kiếm thông tin trên Google về các sự kiện hiện tại hoặc dữ liệu thực tế." # Useful when you need to search for information on Google about current events or factual data.
)

# Công cụ tính toán
calculator_tool = Calculator()

# Custom Tool: Tra cứu thời tiết giả lập (có thể thay bằng API thật nếu có)
@st.cache_data(ttl=3600) # Cache kết quả của tool này trong 1 giờ
def get_weather_data(location: str) -> str:
    """
    Lấy thông tin thời tiết hiện tại cho một địa điểm cụ thể.
    Đầu vào là tên thành phố hoặc địa điểm (ví dụ: "Hà Nội", "Đà Nẵng", "Tokyo").
    Trả về chuỗi mô tả thời tiết.
    """
    location_lower = location.lower()
    if "hà nội" in location_lower: # Hanoi
        return "Thời tiết Hà Nội: 28°C, trời nắng, độ ẩm 70%, gió nhẹ." # Hanoi Weather: 28°C, sunny, 70% humidity, light wind.
    elif "đà nẵng" in location_lower: # Da Nang
        return "Thời tiết Đà Nẵng: 30°C, trời quang đãng, độ ẩm 65%, không có mưa." # Da Nang Weather: 30°C, clear sky, 65% humidity, no rain.
    elif "london" in location_lower:
        return "Thời tiết London: 15°C, nhiều mây, có mưa phùn, gió mạnh." # London Weather: 15°C, cloudy, light drizzle, strong wind.
    elif "tokyo" in location_lower:
        return "Thời tiết Tokyo: 25°C, nắng đẹp, độ ẩm 60%." # Tokyo Weather: 25°C, sunny, 60% humidity.
    else:
        return f"Không tìm thấy thông tin thời tiết cho địa điểm: {location}." # Weather information not found for location:

weather_tool = Tool(
    name="Weather Lookup",
    func=get_weather_data,
    description="Hữu ích khi bạn cần biết thông tin thời tiết hiện tại của một thành phố cụ thể." # Useful when you need to know the current weather information for a specific city.
)

tools = [search_tool, calculator_tool, weather_tool]

# --- 3. Thiết lập Memory cho Agent ---
# Sử dụng ConversationBufferMemory để duy trì lịch sử trò chuyện
# memory_key phải khớp với variable_name trong MessagesPlaceholder của prompt
# return_messages=True để lịch sử được trả về dưới dạng danh sách các đối tượng Message
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# --- 4. Định nghĩa Prompt cho Agent ---
# Prompt này hướng dẫn LLM cách suy nghĩ và sử dụng các công cụ.
# MessagesPlaceholder(variable_name="chat_history") là nơi lịch sử trò chuyện được chèn vào.
# MessagesPlaceholder(variable_name="agent_scratchpad") là nơi Agent ghi lại các Thought, Action, Observation.
prompt = ChatPromptTemplate.from_messages([
    ("system", "Bạn là một trợ lý đa chức năng hữu ích. Bạn có quyền truy cập vào các công cụ sau: {tools}. Sử dụng chúng để trả lời các câu hỏi của người dùng. Hãy duy trì ngữ cảnh cuộc trò chuyện và trả lời một cách tự nhiên."), # You are a helpful multi-functional assistant. You have access to the following tools: {tools}. Use them to answer user questions. Maintain conversation context and respond naturally.
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# --- 5. Tạo Agent và Agent Executor ---
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory, # Truyền đối tượng memory vào Agent Executor
    verbose=True, # Bật verbose để xem quá trình suy nghĩ của Agent trong console
    handle_parsing_errors=True # Cho phép Agent cố gắng tự sửa lỗi phân tích cú pháp
)

# --- 6. Giao diện người dùng Streamlit ---
st.set_page_config(page_title="Chatbot Đa Chức Năng", page_icon="🤖") # Multi-functional Chatbot
st.title("🤖 Chatbot Đa Chức Năng") # Multi-functional Chatbot

# Khởi tạo lịch sử trò chuyện trong session state nếu chưa có
if "messages" not in st.session_state:
    st.session_state.messages = []

# Hiển thị lịch sử trò chuyện
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# Xử lý đầu vào của người dùng
if user_query := st.chat_input("Bạn muốn hỏi gì?"): # What do you want to ask?
    # Thêm tin nhắn của người dùng vào lịch sử
    st.session_state.messages.append({"role": "user", "content": user_query})
    with st.chat_message("user"):
        st.markdown(user_query)

    with st.chat_message("assistant"):
        with st.spinner("Đang suy nghĩ..."): # Thinking...
            try:
                # Gọi Agent Executor với câu hỏi của người dùng
                # Agent Executor sẽ tự động lấy lịch sử từ 'memory' đã được truyền vào
                response = agent_executor.invoke({"input": user_query})
                ai_response = response["output"]
            except Exception as e:
                ai_response = f"Xin lỗi, có lỗi xảy ra: {e}. Vui lòng thử lại hoặc hỏi câu hỏi khác." # Sorry, an error occurred: {e}. Please try again or ask another question.
                st.error(ai_response)
        
        st.markdown(ai_response)
        # Thêm phản hồi của AI vào lịch sử
        st.session_state.messages.append({"role": "assistant", "content": ai_response})

# Nút để xóa lịch sử trò chuyện
if st.button("Xóa lịch sử trò chuyện"): # Clear chat history
    st.session_state.messages = []
    memory.clear() # Xóa lịch sử trong LangChain memory
    st.experimental_rerun() # Chạy lại ứng dụng để cập nhật giao diện
